## Plagiarism Code

#### This code compares documents word by word to detect plagiarism. This method is called longest string comparision as the code tries to find the commom longest strings in the document. the length os this string can be change, currently it is min 2 words and maximum 25.

#### Finally, the code outputs HTML files that show comparisions between two consecutive documents at a time. The HTML files are linked and can be moved 'Next' or 'Previous' to go to next time frame or prevois one respectively.

In [1]:
# Necessary imports
import os
import glob
import re
from itertools import combinations

In [10]:
 # Enter the company ticker you want to do the analysis for
company_name = 'aapl'
# Import the downloaded risk factors file
all_file = glob.glob('risk_factors/'+company_name+'*')

# Function to get names of the files
def getLabel(path):
    sp =path.split('.')
    sp1 =sp[0].split('\\')
    return sp1[1]

docLabels = [getLabel(name) for name in all_file ]

#  Read all files
doc = []
for file in all_file:
    f  = open(file,'r',encoding='utf-8')
    f1 = f.read()
    doc.append(f1)
    f.close()


In [17]:
# preprocess and covert to lower case
def wordList(document): 
    return re.findall("(\w+|\d+)",document.lower())

## Function definition to compare the documents, sentence by sentence 
##~ can change the min and max number of words to be considered for a sentence

def compareDocs(documents, minSize=2, maxSize=25):
    result  = dict() # { (documentIndex,documentIndex) : [CommonExpressions] }
    def tallyDuplicates(expressionDocs):
        for expression,docIndexes in expressionDocs.items():
            combs= [(n,n+1) for n in range(len(docIndexes)-1)]
            for docIndex,otherDoc in combs:
                result.setdefault((docIndex,otherDoc),[]).append(expression)

    documentWords    = [ wordList(document) for document in documents ]
    wordCounts       = [ len(words) for words in documentWords ]
    expressionRanges = dict()
    for docIndex,words in enumerate(documentWords):
        for wordIndex,word in enumerate(words):
            expressionRanges.setdefault(word,[]).append((docIndex,wordIndex))
            
    size = 1
    while size == 1 or expressionDocs and size <= maxSize:        
        nextExpressions   = dict()
        expressionDocs    = dict()
        for expression,starts in expressionRanges.items():
            for docIndex,startIndex in starts:
                endIndex = startIndex+size
                if endIndex >= wordCounts[docIndex]: continue
                extended = " ".join([expression,documentWords[docIndex][endIndex]])
                expressionDocs.setdefault(extended,set()).add(docIndex)
                nextExpressions.setdefault(extended,[]).append( (docIndex,startIndex) )
        expressionDocs   = { expression:docIndexes for expression,docIndexes in expressionDocs.items() if len(docIndexes) > 1 }
        expressionRanges = { expression:ranges for expression,ranges in nextExpressions.items() if expression in expressionDocs }  
        if size >= minSize: tallyDuplicates(expressionDocs)
        size += 1
    return result


In [15]:
## Function definition for the analysis of comparision and formatting for the HTML 
# in terms of perfecentage of files being similar

def analyzeComparison(doc1,doc2,commonExpr):
    words1  = wordList(doc1)
    words2  = wordList(doc2)
    normalizedDoc1 = " ".join(words1)
    normalizedDoc2 = " ".join(words2)
    expressions.sort(key=lambda s:len(s),reverse=True)
    matches = []
    for expression in expressions:
        count1 = len(re.findall(expression,normalizedDoc1))
        count2 = len(re.findall(expression,normalizedDoc2))
        commonCount = min(count1,count2)
        if commonCount == 0: continue
        expressionId = "<#"+str(len(matches))+"#>"
        normalizedDoc1 = re.sub(expression,expressionId,normalizedDoc1,commonCount)
        normalizedDoc2 = re.sub(expression,expressionId,normalizedDoc2,commonCount)
        matches.append((expression,commonCount))
    commonWords = sum( count*len(expr.split(" ")) for expr,count in matches)
    percent1 = 100*commonWords/len(words1)
    percent2 = 100*commonWords/len(words2)
    for index,match in enumerate(matches):
        expressionId = "<#"+str(index)+"#>"
        expressionHighlight = "<span style='background-color:white'>"+match[0]+"</span>"
        normalizedDoc1 = re.sub(expressionId,expressionHighlight,normalizedDoc1)
        normalizedDoc2 = re.sub(expressionId,expressionHighlight,normalizedDoc2)
    return (percent1,percent2,matches,normalizedDoc1,normalizedDoc2)

#### Main Method

In [18]:
## Code to call the comparision and outout result as HTML Page ~ Main code

documents   = doc  # [doc1,doc2,doc3]
comparisons = compareDocs( documents )

for documentPair,expressions in comparisons.items():
    docIndex1,docIndex2 = documentPair
    doc1 = documents[docIndex1]
    doc2 = documents[docIndex2]        
    pct1,pct2,matches,doc1,doc2 = analyzeComparison(doc1,doc2,expressions)

    # print result on console ...
    print(int(pct1//1)," % of document #",docLabels[docIndex1]," is same as document #", docLabels[docIndex2])
    print(int(pct2//1)," % of document #",docLabels[docIndex2]," is same as document #", docLabels[docIndex1])
    print("Common expressions are:")
    for expression,count in matches:
        print( "    ",expression,"(",count,"times )")
    print("")

    # output comparison result to an HTML file...
    if docIndex1 == 0:
        htmlPage = "<html><body><p style='text-align:left;'><a  align='right' href = '"+str(docLabels[docIndex2])+"-"+str(docLabels[docIndex2+1])+".html'>Next</a></p> <table border='1'>"
    elif docIndex2 == (len(docLabels)-1):
        htmlPage = "<html><body><p style='text-align:left;'><a  align='right' href = '"+str(docLabels[docIndex1-1])+"-"+str(docLabels[docIndex1])+".html'>Previous</a></p> <table border='1'>"
    else:
        htmlPage = "<html><body><p style='text-align:left;'><a  align='right' href = '"+str(docLabels[docIndex1-1])+"-"+str(docLabels[docIndex1])+".html'>Previous</a></p><p style='text-align:left;'><a  align='right' href = '"+str(docLabels[docIndex2])+"-"+str(docLabels[docIndex2+1])+".html'>Next</a></p> <table border='1'>"                 
    htmlPage += "<tr><th>#" + str(docLabels[docIndex1]) + ": Source " + str(int(pct1//1)) + "% duplicate</th>"
    htmlPage += "<th>#" + str(docLabels[docIndex2]) + ": Target  " + str(int(pct2//1)) + "% duplicate</th></tr>"
    htmlPage += "<tr><td width='50%' valign='top'>" + "<span style='background-color:yellow'>"+doc1+"</span>" + "</td><td valign='top'>" + "<span style='background-color:yellow'>"+doc2+"</span>" + "</td></tr>"
    htmlPage +="</table></body></html>"        
    fileName = str(docLabels[docIndex1])+"-"+str(docLabels[docIndex2])+".html"
    with open(fileName,"w") as f: f.write(htmlPage) 

98  % of document # aapl_20180202_10q  is same as document # aapl_20180502_10q
98  % of document # aapl_20180502_10q  is same as document # aapl_20180202_10q
Common expressions are:
     services telecommunications mobile communications and media television intellectual property ownership and infringement tax import and export requirements anti corruption foreign exchange controls and cash repatriation restrictions ( 1 times )
     releases of confidential information including personally identifiable information that could subject the company to significant reputational financial legal and operational consequences the company s business requires ( 1 times )
     business strategies or acquisitions such endeavors may involve significant risks and uncertainties including distraction of management from current operations greater than expected liabilities and expenses inadequate ( 1 times )
     frequent introduction of new products short product life cycles evolving industry standards co

     risk or other factors as a result the value and liquidity of the company s cash cash equivalents ( 1 times )
     by competitors and price sensitivity on the part of consumers the company s ability to compete ( 1 times )
     does not obligate it to acquire any specific number of shares if the company fails to meet ( 1 times )
     of these partners to perform may have a negative impact on the company s cost or supply of ( 1 times )
     the company also sells its products and third party products in most of its major markets ( 1 times )
     to buy the company s products with respect to its mac products the company believes the ( 1 times )
     time to time that can result in decreased sales and operating margin and harm to the ( 1 times )
     2018 form 10 q the company depends on the performance of distributors carriers and ( 1 times )
     revenue that may be generated expected future growth of mac sales and the costs of ( 1 times )
     condition and operating results there m

94  % of document # aapl_20180502_10q  is same as document # aapl_20180801_10q
91  % of document # aapl_20180801_10q  is same as document # aapl_20180502_10q
Common expressions are:
     services telecommunications mobile communications and media television intellectual property ownership and infringement tax import and export requirements anti corruption foreign exchange controls and cash repatriation restrictions ( 1 times )
     releases of confidential information including personally identifiable information that could subject the company to significant reputational financial legal and operational consequences the company s business requires ( 1 times )
     business strategies or acquisitions such endeavors may involve significant risks and uncertainties including distraction of management from current operations greater than expected liabilities and expenses inadequate ( 1 times )
     frequent introduction of new products short product life cycles evolving industry standards co

     economic business labor environmental public health or political issues ( 1 times )
     pii affect many companies across various industries the company is at a ( 1 times )
     but not limited to those described below any one or more of which could ( 1 times )
     on sales of the company s products in foreign countries and on sales of ( 1 times )
     of which have broader product lines lower priced products and a larger ( 1 times )
     failure of derivative counterparties and other financial institutions ( 1 times )
     its products directly to small and mid sized businesses and education ( 1 times )
     amounts of components or products or not fully utilize firm purchase ( 1 times )
     in components and products the company purchases from third parties ( 1 times )
     of new products and disruptions in the operations of the company s ( 1 times )
     experience significant expenditures in order to resume operations ( 1 times )
     from time to time experience outages se

91  % of document # aapl_20180801_10q  is same as document # aapl_20181105_10k
92  % of document # aapl_20181105_10k  is same as document # aapl_20180801_10q
Common expressions are:
     services telecommunications mobile communications and media television intellectual property ownership and infringement tax import and export requirements anti corruption foreign exchange controls and cash repatriation restrictions ( 1 times )
     terrorism natural disasters public health issues industrial accidents and other business interruptions political events international trade disputes war terrorism natural disasters public health issues industrial accidents ( 1 times )
     suppliers contract manufacturers logistics providers distributors cellular network carriers and other channel partners potential effects include financial instability inability to obtain credit to finance operations and ( 1 times )
     customers suppliers contract manufacturers logistics providers distributors cellular ne

...

     company s cost of ( 1 times )
     of this form 10 q ( 1 times )
     the company also ( 1 times )
     for its products ( 3 times )
     gross margins on ( 2 times )
     of the company s ( 2 times )
     if the company s ( 1 times )
     the company has ( 2 times )
     of new products ( 1 times )
     also exposed to ( 1 times )
     in the notes to ( 1 times )
     and cash flows ( 1 times )
     or channel mix ( 1 times )
     are subject to ( 1 times )
     to the risk of ( 1 times )
     in part on the ( 1 times )
     or errors as a ( 1 times )
     the company s ( 4 times )
     also exist in ( 1 times )
     sales and the ( 1 times )
     fees or fines ( 1 times )
     in the mix of ( 1 times )
     form 10 q the ( 1 times )
     2018 form 10 ( 1 times )
     2019 form 10 ( 5 times )
     to fuel and ( 1 times )
     are in high ( 1 times )
     some of the ( 1 times )
     and use of ( 1 times )
     if at all ( 1 times )
     i item 1 ( 1 times )
     the u s ( 1 times

     could be negatively impacted the company s products and services may ( 1 times )
     conversely a strengthening of foreign currencies relative to the u s ( 1 times )
     in components and products the company purchases from third parties ( 1 times )
     on the quality or quantity of products or services or the company s ( 1 times )
     these and other economic factors could materially adversely affect ( 1 times )
     of new products and disruptions in the operations of the company s ( 1 times )
     experience significant expenditures in order to resume operations ( 1 times )
     of the supplier code of conduct could occur the company relies on ( 1 times )
     from time to time experience outages service slowdowns or errors ( 1 times )
     sales and the costs of developing such applications and services ( 1 times )
     the company has a minority market share in the global smartphone ( 1 times )
     the outcome of litigation is inherently uncertain if one or more ( 1 time

     are in locations that ( 1 times )
     through the app store ( 1 times )
     to perform may have a ( 1 times )
     adversely affect the ( 1 times )
     products or services ( 1 times )
     net realizable value ( 1 times )
     natural disasters in ( 1 times )
     public health issues ( 2 times )
     the company s retail ( 1 times )
     by the company often ( 1 times )
     gross margins on the ( 1 times )
     to buy the company s ( 1 times )
     taxes in the u s and ( 1 times )
     material changes to ( 1 times )
     to provide products ( 1 times )
     are located outside ( 1 times )
     and the company may ( 1 times )
     could be subject to ( 1 times )
     while the company s ( 1 times )
     it to use and store ( 1 times )
     s manufacturing is ( 1 times )
     could increase the ( 1 times )
     service for iphone ( 1 times )
     devices may suffer ( 1 times )
     the company may be ( 1 times )
     be able to develop ( 1 times )
     the company sells ( 2 t

     reasonable prices with acceptable usage rules or continue to ( 1 times )
     complex hardware and software products and services that can ( 1 times )
     or otherwise have a material adverse impact on the company s ( 1 times )
     for periods up to 150 days because the company s markets are ( 1 times )
     financial condition and operating results while the company ( 1 times )
     some carriers providing cellular network service for iphone ( 1 times )
     the company currently holds a significant number of patents ( 1 times )
     may suffer from poor financial conditions which can lead to ( 1 times )
     amounts of components or products or not fully utilize firm ( 1 times )
     which among other things may result in the delay or loss of ( 1 times )
     subject the company to costs and damages in the event of a ( 1 times )
     ability to mitigate such risks may be limited the company ( 1 times )
     exchange rates the use of such hedging activities may not ( 1 times )


     net sales in ( 1 times )
     the risk of ( 1 times )
     the cost of ( 1 times )
     and may not ( 1 times )
     as a result ( 1 times )
     some of the ( 1 times )
     as well as ( 1 times )
     form 10 k ( 1 times )
     or in the ( 1 times )
     the u s ( 3 times )
     in r d ( 1 times )

1  % of document # aapl_20191031_10k  is same as document # aapl_20200129_10q
79  % of document # aapl_20200129_10q  is same as document # aapl_20191031_10k
Common expressions are:
     directly or indirectly cause the company s actual financial condition and operating results to vary materially from past or from anticipated future financial condition and operating ( 1 times )
     business financial condition and operating results of the company can be affected by a number of factors whether currently known or unknown including but not limited ( 1 times )
     results any of these factors in whole or in part could materially and adversely affect the company s business financial condi

#### Credits: 
##### GeeksforGeeks: https://www.geeksforgeeks.org/longest-common-substring-dp-29/ 
##### Stackoverflow : https://stackoverflow.com/questions/8897593/how-to-compute-the-similarity-between-two-text-documents
#####               https://stackoverflow.com/questions/54886961/whats-the-best-algorithm-to-check-the-similarity-percentage-among-the-submitted
#####            https://stackoverflow.com/questions/20708309/approach-to-longest-common-substring 